In [1]:
import sys
#sys.path.append('/afs/crc.nd.edu/user/d/dsimone2/cctools/lib/python3.8/site-packages')
import work_queue as wq
import ipywidgets as widgets
import time

num_tasks = widgets.IntSlider(
    value=100,
    min=20,
    max=1000,
    step=1,
    description='Number of tasks:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
num_workers = widgets.IntSlider(
    value=10,
    min=5,
    max=30,
    step=1,
    description='Number of tasks:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(num_tasks)
display(num_workers)

IntSlider(value=100, continuous_update=False, description='Number of tasks:', max=1000, min=20)

IntSlider(value=10, continuous_update=False, description='Number of tasks:', max=30, min=5)

In [ ]:
#!/usr/bin/env python

# Copyright (c) 2010- The University of Notre Dame.
# This software is distributed under the GNU General Public License.
# See the file COPYING for details.

PORT = 0
number_of_tasks = num_tasks.value
starting_number = 2

def is_prime(number):
    import math
    time.sleep(10)
    for i in range(2, int(math.sqrt(number)) + 1):
            if (number % i == 0):
                return 0
    return 1

horizontal_boxes = 20
vertical_boxes = number_of_tasks // horizontal_boxes + 1

def display_output(prime):
    vertical_temp = []
    for j in range(vertical_boxes):
        horizontal_temp = []
        for i in range(horizontal_boxes):
            if (i + j * horizontal_boxes + 2) > number_of_tasks:
                break
            horizontal_temp.append(prime[i + j * horizontal_boxes + 2])
        vertical_temp.append(widgets.HBox(horizontal_temp))
    output = widgets.VBox(vertical_temp)
    display(output)


prime = []
for i in range(starting_number, starting_number + number_of_tasks + num_workers.value + 10):
    prime.append(widgets.ToggleButton(
        value=False,
        description=str(i - 2),
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        layout=widgets.Layout(width='20%', height='20px')
    ))
    
tasks_done_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=number_of_tasks,
    description='Number of tasks complete:',
    style = {'description_width': 'initial'},
    orientation='horizontal'
)
tasks_idling = widgets.IntProgress(
    value=0,
    min=0,
    max=5,
    description='Number of workers idle',
    style = {'description_width': 'initial'},
    orientation='horizontal'
)
workers_connected = widgets.IntProgress(
    value=0,
    min=0,
    max=num_workers.value,
    description='Number of workers connected',
    style = {'description_width': 'initial'},
    orientation='horizontal'
)
worker_properties = ["Number of workers", "Cores", "Memory", "Disk", "GPUs"]
worker_array = []
for name in worker_properties:
    worker_array.append(widgets.Text(description=name, layout=widgets.Layout(width='15%', height='40px')))
display(widgets.HBox([tasks_done_bar, workers_connected, tasks_idling]))
display_output(prime)
display(widgets.HBox(worker_array))

q = wq.WorkQueue(PORT)
workers = wq.Factory('local', manager_host_port=f'localhost:{q.port}')
workers.cores = 1
workers.memory = 100
workers.disk = 100
workers.min_workers = 10
workers.max_workers = num_workers.value

for i in range(2, number_of_tasks + 1):
    p_task = wq.PythonTask(is_prime, i)
    p_task.specify_environment("my-minimal-env.tar.gz")
    q.submit(p_task)

with workers:
    while not q.empty():
        t = q.wait(1)
        if t:
            x = t.output
            if x == 1:
                prime[int(t.id) + 1].disabled = True
            prime[int(t.id) + 1].button_style = 'success'
            for i in range(q.stats.workers_connected + 1):
                if (int(t.id) + 1 - i) > 0:
                    if (prime[int(t.id) + 1 - i].button_style == ''):
                        prime[int(t.id) + 1 - i].button_style='warning'
        tasks_done_bar.value = q.stats.tasks_done
        tasks_idling.value = q.stats.workers_idle
        workers_connected.value = q.stats.workers_connected
        print(q.stats.workers_connected)
        #worker_array[0].value = str(q.workers_summary()[0]['workers'])
        #worker_array[1].value = str(q.workers_summary()[0]['cores'])
        #worker_array[2].value = str(q.workers_summary()[0]['memory'])
        #worker_array[3].value = str(q.workers_summary()[0]['disk'])
        #worker_array[4].value = str(q.workers_summary()[0]['gpus'])
                    
                    
#q.stats.tasks_done                
print("all tasks complete!")

5
5
5
5
5
5
5
5
5
5
5
5
5
